# CHECK COMPUTER AVAILABILITY

In [1]:
# Check GPU
!nvidia-smi

# Check CPU and RAM
!lscpu
!free -h

# Check disk space
!df -h

# Check details with psutil
import psutil

# CPU information
print("CPU:", psutil.cpu_percent(), "%")
print("Physical cores:", psutil.cpu_count(logical=False))
print("Logical cores:", psutil.cpu_count(logical=True))

# RAM information
mem = psutil.virtual_memory()
print("Total RAM:", mem.total / (1024 ** 3), "GB")
print("Available RAM:", mem.available / (1024 ** 3), "GB")

# Disk information
disk = psutil.disk_usage('/')
print("Total disk space:", disk.total / (1024 ** 3), "GB")
print("Used disk space:", disk.used / (102

Fri Jan 31 22:28:41 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   39C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

# DOWNLOAD REQUERIMENTS



In [1]:
# prompt: install python requirements

!pip install -r requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.5/181.5 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 914.9/914.9 kB 55.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 99.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 81.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 55.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 65.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-

# EXECUTING CODE

In [2]:
import numpy as np
import supervision as sv
from ultralytics import YOLO
import argparse

# Instead of parsing command-line arguments, define them directly:
input_video_path = '/content/demo.mp4'  # Replace with your input video path
output_video_path = '/content/output_video.mp4' # Replace with your desired output path

class CountObject():

    def __init__(self,input_video_path,output_video_path) -> None:

        self.model = YOLO('yolov8s.pt')
        self.colors = sv.ColorPalette.from_hex(['#FF0000', '#00FF00', '#0000FF'])

        self.input_video_path = input_video_path
        self.output_video_path = output_video_path

        self.polygons = [
            np.array([
                [540,  985 ],
                [1620, 985 ],
                [2160, 1920],
                [1620, 2855],
                [540,  2855],
                [0,    1920]
            ], np.int32),
            np.array([
                [0,    1920],
                [540,  985 ],
                [0,    0   ]
            ], np.int32),
            np.array([
                [1620, 985 ],
                [2160, 1920],
                [2160,    0]
            ], np.int32),
            np.array([
                [540,  985 ],
                [0,    0   ],
                [2160, 0   ],
                [1620, 985 ]
            ], np.int32),
            np.array([
                [0,    1920],
                [0,    3840],
                [540,  2855]
            ], np.int32),
            np.array([
                [2160, 1920],
                [1620, 2855],
                [2160, 3840]
            ], np.int32),
            np.array([
                [1620, 2855],
                [540,  2855],
                [0,    3840],
                [2160, 3840]
            ], np.int32)
        ]

        #self.video_info = sv.VideoInfo.from_video_path(input_video_path) # This line is no longer needed
        self.zones = [
            sv.PolygonZone(
                polygon=polygon,
                #frame_resolution_wh=self.video_info.resolution_wh # Remove this argument
            )
            for polygon
            in self.polygons
        ]

        self.zone_annotators = [
            sv.PolygonZoneAnnotator(
                zone=zone,
                color=self.colors.by_idx(index),
                thickness=6,
                text_thickness=8,
                text_scale=4
            )
            for index, zone
            in enumerate(self.zones)
        ]

        self.box_annotators = [
            sv.BoxAnnotator(
                color=self.colors.by_idx(index),
                thickness=4,
            )
            for index
            in range(len(self.polygons))
        ]

    def process_frame(self,frame: np.ndarray, i) -> np.ndarray:
        # detect
        results = self.model(frame, imgsz=1280)[0]
        detections = sv.Detections.from_ultralytics(results)
        detections = detections[(detections.class_id == 0) & (detections.confidence > 0.5)]

        for zone, zone_annotator, box_annotator in zip(self.zones, self.zone_annotators, self.box_annotators):
            mask = zone.trigger(detections=detections)
            detections_filtered = detections[mask]
            frame = box_annotator.annotate(scene=frame, detections=detections_filtered)
            frame = zone_annotator.annotate(scene=frame)

        return frame

    def process_video(self):

        sv.process_video(source_path=self.input_video_path, target_path=self.output_video_path, callback=self.process_frame)


if __name__ == "__main__":
    obj = CountObject(input_video_path, output_video_path)
    obj.process_video()

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


100%|██████████| 21.5M/21.5M [00:00<00:00, 379MB/s]



0: 736x1280 44 persons, 1 backpack, 4 handbags, 1 tennis racket, 53.9ms
Speed: 26.0ms preprocess, 53.9ms inference, 380.8ms postprocess per image at shape (1, 3, 736, 1280)

0: 736x1280 44 persons, 2 backpacks, 5 handbags, 1 tennis racket, 30.0ms
Speed: 12.4ms preprocess, 30.0ms inference, 1.6ms postprocess per image at shape (1, 3, 736, 1280)

0: 736x1280 42 persons, 1 backpack, 5 handbags, 1 tennis racket, 30.0ms
Speed: 11.3ms preprocess, 30.0ms inference, 1.7ms postprocess per image at shape (1, 3, 736, 1280)

0: 736x1280 43 persons, 6 backpacks, 4 handbags, 1 tennis racket, 30.0ms
Speed: 12.2ms preprocess, 30.0ms inference, 1.7ms postprocess per image at shape (1, 3, 736, 1280)

0: 736x1280 42 persons, 5 backpacks, 5 handbags, 1 tennis racket, 30.0ms
Speed: 11.3ms preprocess, 30.0ms inference, 2.4ms postprocess per image at shape (1, 3, 736, 1280)

0: 736x1280 41 persons, 4 backpacks, 6 handbags, 1 tennis racket, 30.0ms
Speed: 9.5ms preprocess, 30.0ms inference, 1.8ms postprocess 

In [3]:
# prompt: dame un código de consola para ver las librerias qué tengo y su version

!pip freeze > actual_requirements.txt
